# 학습 데이터로 앙상블 학습

In [0]:
# 구글 드라이브와 연결
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
# 모듈 삽입
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt 
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

# 드라이브에 저장된 데이터 불러오기
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/co_data/digit-recognizer/train.csv')
test_data = np.array(pd.read_csv('/content/drive/My Drive/Colab Notebooks/co_data/digit-recognizer/test.csv'))


# 그래프 초기화
tf.reset_default_graph()


# train data 설정
x_data = np.array(train_data.drop(['label'], axis='columns', inplace=False))
y_data = np.array(train_data['label'])
y_data = pd.get_dummies(train_data.label).values
y_data.shape


# 정규화
#scaler=MinMaxScaler()
#x_data = scaler.fit_transform(x_data)


# 데이터 구분
split_num = int(x_data.shape[0]*0.8)
train_x_data = x_data[:split_num]
test_x_data = x_data[split_num:]
train_y_data = y_data[:split_num]
test_y_data = y_data[split_num:]


# Placeholder
X = tf.placeholder(shape = [None, 784], dtype=tf.float32)
Y = tf.placeholder(shape = [None, 10], dtype=tf.float32)
drop_rate = tf.placeholder(dtype=tf.float32)


# image data 설정
x_img = tf.reshape(X, [-1,28,28,1])



ModuleNotFoundError: ignored

In [0]:
# CNN
L1 = tf.layers.conv2d(inputs = x_img,
                      filters = 32,
                      kernel_size = [3,3],
                      padding ="SAME",
                      strides = 1, 
                      activation = tf.nn.relu)
L1 = tf.layers.max_pooling2d(inputs = L1,
                             pool_size = [2,2],
                             padding = "SAME",
                             strides = 2)


L2 = tf.layers.conv2d(inputs = L1,
                      filters = 64,
                      kernel_size = [3,3],
                      padding ="SAME",
                      strides = 1,
                      activation = tf.nn.relu)
L2 = tf.layers.max_pooling2d(inputs = L2,
                             pool_size = [2,2],
                             padding = "SAME",
                             strides = 2)


convoled_data = tf.reshape(L2, [-1,7*7*64])


# DF
layer1 = tf.layers.dense(inputs = convoled_data,
                         units = 256,
                         activation = tf.nn.relu)
layer1 = tf.layers.dropout(inputs = layer1,
                           rate = drop_rate)
layer2 = tf.layers.dense(inputs = layer1,
                         units = 128,   
                         activation = tf.nn.relu)
layer2 = tf.layers.dropout(inputs = layer2,
                           rate = drop_rate)
layer3 = tf.layers.dense(inputs = layer2,
                         units = 512,  
                         activation = tf.nn.relu)
layer3 = tf.layers.dropout(inputs = layer2,
                           rate = drop_rate)


H = tf.layers.dense(inputs = layer3, units=10)

cost = tf.losses.softmax_cross_entropy(Y, H)

train = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# 앙상블 학습
for j in range(10):
    
    train_epoch = 30
    batch_size = 100

    tf.reset_default_graph()

    for step in range(train_epoch):
        num_of_iter = int(train_x_data.shape[0] / batch_size)

        for i in range(num_of_iter):
            batch_x = train_x_data[i*batch_size:(i+1)*batch_size]
            batch_y = train_y_data[i*batch_size:(i+1)*batch_size]

            _, cost_val = sess.run([train, cost],
                                   feed_dict={X: batch_x,
                                              Y: batch_y,
                                              drop_rate: 0.3})
        if step % 5 == 0:
          print(cost_val)
    
    H_val = sess.run(H, feed_dict={X: test_x_data,drop_rate : 0 })
    
    if j == 0:
      H_result = H_val
    else:
      H_result = np.add(H_result,H_val)
        

0.09461319
0.030516401
0.02125607
0.013009138


KeyboardInterrupt: ignored

In [0]:
# print(np.argmax(H_result,1))
# https://gomguard.tistory.com/145
print(test_y_data)

In [0]:
predict = np.argmax(H_result,1)
correct = np.equal(predict, np.argmax(test_y_data,1))
accuracy = np.mean(correct)
print(accuracy)

In [0]:
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print(f'정확도:{sess.run(accuracy, feed_dict = {X: test_x_data, Y: test_y_data})}')